In [1]:
import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
import tensorflow.keras.backend as K
from commai_spd_chl_keras_datagen import commai_speed_chl_gen
from tensorflow.keras.layers import Dense,Activation,Lambda,GRU,BatchNormalization,Activation,Flatten,Conv2D,GlobalMaxPooling2D,MaxPooling2D,Reshape,Dropout

Using TensorFlow backend.


In [2]:
tf.config.list_physical_devices ()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.keras.backend.image_data_format()

'channels_last'

In [5]:
seq = Sequential()

#should I trust this guy?https://www.kdnuggets.com/2018/09/dropout-convolutional-networks.html

seq.add(Lambda(lambda x:tf.divide(x,255),input_shape=(480*2,640,1))) #scale the inputs between 0-1

seq.add(Conv2D(filters=64,strides=(2,2),kernel_initializer='he_normal', kernel_size=(4,4),padding='same'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, strides=(2,2),kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64,strides=(2,2), kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=96,strides=(2,2), kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=96,strides=(2,2), kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=2,strides=(2,2), kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())


seq.add(GlobalMaxPooling2D())

seq.add(Dense(1))

seq.compile(loss='mse', optimizer='adam')
seq.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 960, 640, 1)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 480, 320, 64)      1088      
_________________________________________________________________
activation_6 (Activation)    (None, 480, 320, 64)      0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 480, 320, 64)      256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 240, 160, 64)      65600     
_________________________________________________________________
activation_7 (Activation)    (None, 240, 160, 64)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 240, 160, 64)     

In [4]:
batch_size=32
n_e=60
n_frames_train=15000
n_frames_test=2900
n_frames_valid=2500

train_idx,test_idx,valid_idx=commai_speed_chl_gen(n_frames_valid=n_frames_valid,frames_per_sample=2,mode="stacked",
                    n_frames_test=n_frames_test,n_frames_train=n_frames_train,batch_size=batch_size).get_indexes()

load=np.load(open("idxes_21bs_STACKED_15k.np","rb"))
train_idx=load["train_idx"]
test_idx=load["test_idx"]
valid_idx=load["valid_idx"]

train_gen=commai_speed_chl_gen(frame_idx=train_idx,batch_size=batch_size,mode="stacked",frames_per_sample=2)
test_gen=commai_speed_chl_gen(frame_idx=test_idx,batch_size=batch_size,mode="stacked",frames_per_sample=2)
valid_gen=commai_speed_chl_gen(frame_idx=valid_idx,batch_size=batch_size,mode="stacked",frames_per_sample=2)

#np.savez_compressed(open("idxes_21bs_STACKED_15k.np","wb+"),train_idx=train_idx,test_idx=test_idx,valid_idx=valid_idx)



(480, 640, 3)
(32, 960, 640)
(32, 2, 1)
train_samples: 7500 
test_samples:1450 
valid_samples: 1250
(480, 640, 3)
(32, 960, 640)
(32, 2, 1)
(480, 640, 3)
(32, 960, 640)
(32, 2, 1)
(480, 640, 3)
(32, 960, 640)
(32, 2, 1)


In [ ]:
ie=0
filepath="STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-FULL_CONV-{epoch:02d}-{val_loss:.2f}-{loss:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False)

tb=tf.keras.callbacks.TensorBoard(
    log_dir='STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-FULL_CONV', histogram_freq=1, write_graph=True, write_images=False,
    update_freq=10, profile_batch=0, embeddings_freq=0
)

cbs=[checkpoint,tb]
hst=seq.fit_generator(train_gen, epochs=n_e,workers=30,max_queue_size=100,
        use_multiprocessing=True,validation_data=test_gen,callbacks=cbs,verbose=1,shuffle=True, initial_epoch=ie)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/60
234/234 [==============================] - ETA: 0s - loss: 66.9689WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-FULL_CONV-01-95.88-66.97.hdf5
234/234 [==============================] - 127s 543ms/step - loss: 66.9689 - val_loss: 95.8752
Epoch 2/60
234/234 [==============================] - ETA: 0s - loss: 45.5457WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-FULL_CONV-02-50.24-45.55.hdf5
234/234 [==============================] - 136s 581ms/step - loss: 45.5457 - val_loss: 50.2365
Epoch 3/60
234/234 [================

234/234 [==============================] - 126s 537ms/step - loss: 11.4409 - val_loss: 11.1189
Epoch 10/60
234/234 [==============================] - ETA: 0s - loss: 11.4707WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00010: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-FULL_CONV-10-10.95-11.47.hdf5
234/234 [==============================] - 133s 570ms/step - loss: 11.4707 - val_loss: 10.9539
Epoch 11/60
234/234 [==============================] - ETA: 0s - loss: 9.5138WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00011: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-FULL_CONV-11-11.13-9.51.hdf5
234/234 [==============================] - 140s 599ms/step - loss: 9.5138 - val_loss: 11.1264
Epoch 12/60
234/23

234/234 [==============================] - ETA: 0s - loss: 5.1859WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00018: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-FULL_CONV-18-6.10-5.19.hdf5
234/234 [==============================] - 132s 564ms/step - loss: 5.1859 - val_loss: 6.1028
Epoch 19/60
234/234 [==============================] - ETA: 0s - loss: 4.5742WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00019: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-FULL_CONV-19-11.76-4.57.hdf5
234/234 [==============================] - 137s 585ms/step - loss: 4.5742 - val_loss: 11.7637
Epoch 20/60
234/234 [==============================] - ETA: 0s - loss: 3.9543WARNING:tensorflow:multiprocessing can interact badly

Process Keras_worker_ForkPoolWorker-2905:
Process Keras_worker_ForkPoolWorker-2799:
Process Keras_worker_ForkPoolWorker-2796:
Process Keras_worker_ForkPoolWorker-2793:
Process Keras_worker_ForkPoolWorker-2818:
Process Keras_worker_ForkPoolWorker-2808:
Process Keras_worker_ForkPoolWorker-2811:
Process Keras_worker_ForkPoolWorker-2909:
Process Keras_worker_ForkPoolWorker-2887:
Process Keras_worker_ForkPoolWorker-2886:
Process Keras_worker_ForkPoolWorker-2888:
Process Keras_worker_ForkPoolWorker-2803:
Process Keras_worker_ForkPoolWorker-2881:
Process Keras_worker_ForkPoolWorker-2814:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-2895:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-2794:
Process Keras_worker_ForkPoolWorker-2797:
Process Keras_worker_ForkPoolWorker-2792:
Process Keras_worker_ForkPoolWorker-2907:
Process Keras_worker_ForkPoolWorker-2798:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-2816:
Process Keras

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _boo

  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    sel

  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 347, in put
    self._writer.send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  Fi

  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/home/q/spd_chl/lib/python3.6/site-packages/tensorflow/python/keras/utils/data_utils.py", line 662, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/usr/lib/python

In [5]:
#22 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-FULL_CONV-22-4.59-3.33.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

Instructions for updating:
Please use Model.evaluate, which supports generators.
40/39 - 7s - loss: 4.8180


4.818037033081055

In [6]:
#15 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-FULL_CONV-15-6.86-6.34.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

40/39 - 7s - loss: 6.6577


6.657745361328125